In [104]:
import sympy
from sympy import init_printing
from sympy import MutableDenseMatrix
init_printing(use_latex='png')

# Use IPython's Latex capabilities to avoid a bug in Sympy's matrix printing
from IPython.display import display, Math, Latex

import numpy as np
import pyamg
from scipy import sparse
from scipy import ndimage
import rasterio as rio
from geoblend.coefficient_matrix import create_coefficient_matrix

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['image.interpolation'] = 'none'
mpl.rcParams['image.cmap'] = 'cubehelix'

%matplotlib inline

np.set_printoptions(linewidth=200)

In [2]:
def display_matrix(m):
    """
    Helper function to display the coefficient
    matrix.
    """
    
    if type(m) == MutableDenseMatrix:
        tex = sympy.latex(m)
    elif type(m) == np.ndarray:
        tex = sympy.latex(sympy.Matrix(mask.astype(np.int8).tolist()))
    else:
        # Crazy-ness to get integer represention ...
        tex = sympy.latex(sympy.Matrix(m.toarray().astype(np.int8).tolist()))
    
    display(Math(tex))

In [74]:
def create_auxiliary_equation(mask):
    """
    Create a sympy representation of the auxiliary equation corresponding
    to the Poisson equation.
    
    This prevents me from tedious, error-proned, hand-written algebra.
    """
    
    height, width = mask.shape
    x_domain = range(width)
    y_domain = range(height)
    
    # Create symbolic variables representing the unknown image
    # and the known gradient that will be imposed on the unknown image.
    
    # Note: One variable per pixel. f and g are pixel-aligned.
    f = [ sympy.symbols(' '.join([ 'f%d%d' % (i, j) for j in y_domain ])) for i in x_domain ]
    g = [ sympy.symbols(' '.join([ 'g%d%d' % (i, j) for j in y_domain ])) for i in x_domain ]
    c = [ sympy.symbols(' '.join([ 'c%d%d' % (i, j) for j in y_domain ])) for i in x_domain ]
    
    # Create the variable that will comprise the auxiliary equation.
    equation = 0
    
    for j in y_domain:
        for i in x_domain:
            
            if mask[j][i] == 0:
                continue
            
            #
            # Define the index of the 4-connected neighbors
            #
            
            # For a pixel p with indices (i, j), the
            # 4-connected neighbors are defined by the
            # surrounding 3x3 pixel grid as:
            #
            # - N - 
            # W p E
            # - S - 
            
            nj, ni = j - 1, i
            sj, si = j + 1, i
            ej, ei = j, i + 1
            wj, wi = j, i - 1
            
            # TODO: This pays no attention to boundaries. The gradient
            #       at the boundary should never be sampled.
            if nj in y_domain:
                equation += (f[i][j] - f[ni][nj] - (g[i][j] - g[ni][nj])) ** 2
            if ei in x_domain:
                equation += (f[i][j] - f[ei][ej] - (g[i][j] - g[ei][ej])) ** 2
            if sj in y_domain:
                equation += (f[i][j] - f[si][sj] - (g[i][j] - g[si][sj])) ** 2
            if wi in x_domain:
                equation += (f[i][j] - f[wi][wj] - (g[i][j] - g[wi][wj])) ** 2
    
    return equation, f, g, c

In [75]:
mask1 = np.array([
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
], dtype=np.uint8)
mask1

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [76]:
mask2 = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
], dtype=np.uint8)
mask2

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [77]:
with rio.drivers():
    with rio.open("test2/20150705_014439_081f_analytic.tif") as src:
        mask3 = src.read(4).astype(np.uint8)
        mask3[np.nonzero(mask3)] = 1
mask3

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [78]:
eq1, f1, g1, c1 = create_auxiliary_equation(mask1)
eq2, f2, g2, c2 = create_auxiliary_equation(mask2)
# eq3, f3, g3, c3 = create_auxiliary_equation(mask3)

In [79]:
def linearize_system(mask, equation, f):
    
    indices = np.nonzero(mask)
    return [
        sympy.diff(equation, f[idx[1]][idx[0]])
        for idx in zip(*indices)
    ]

In [80]:
system1 = linearize_system(mask1, eq1, f1)
system2 = linearize_system(mask2, eq2, f2)
# system3 = linearize_system(mask3, eq3, f3)

In [81]:
mat1 = sympy.Matrix(system1)
mat2 = sympy.Matrix(system2)
# mat3 = sympy.Matrix(system3)

In [82]:
# display_matrix(mat1)
# display_matrix(mat2)
# display_matrix(mat3)

In [83]:
def get_boundary_mask(m):
    selem = np.array([
        [0, 1, 0],
        [1, -4, 1],
        [0, 1, 0]
    ])
    b = ndimage.convolve(m, selem, mode='constant', cval=0)
    b[b < 0] = 0
    return b

In [84]:
# get_boundary_mask(mask1)

In [85]:
# get_boundary_mask(mask2)

In [86]:
# get_boundary_mask(mask3.astype(np.int8))

In [106]:
def get_symbolic_replacements(mask, f, g, c):
    
    # Replace boundaries with symbolic values
    boundary = get_boundary_mask(mask)
    bindices = np.nonzero(boundary)
    
    return [
        (f[idx[1]][idx[0]], c[idx[1]][idx[0]])
        for idx in zip(*bindices)
    ]

In [107]:
symbolic_replacements1 = get_symbolic_replacements(mask1, f1, g1, c1)
symbolic_replacements2 = get_symbolic_replacements(mask2, f2, g2, c2)
# symbolic_replacements3 = get_symbolic_replacements(mask3, f3, g3, c3)

In [108]:
print mask1
display_matrix(mat1.subs(symbolic_replacements1))
# display_matrix(mat2.subs(symbolic_replacements2))
# display_matrix(mat3.subs(symbolic_replacements3))

[[0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0]]


<IPython.core.display.Math object>

In [90]:
def get_unknown_vector(mask, f):
    
    indices = np.nonzero(mask)
    return [
        f[idx[1]][idx[0]]
        for idx in zip(*indices)
    ]

In [91]:
vec1 = get_unknown_vector(mask1, f1)
vec2 = get_unknown_vector(mask2, f2)
# vec3 = get_unknown_vector(mask3, f3)

In [92]:
coeff1 = mat1.jacobian(vec1)
coeff2 = mat2.jacobian(vec2)
# coeff3 = mat3.jacobian(vec3)

In [93]:
# display_matrix(coeff1)
# display_matrix(coeff2)
# display_matrix(coeff3)
coeff1

Matrix([
[12, -4,  0,  0,  0,  0,  0,  0, -4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[-4, 14, -4,  0,  0,  0,  0,  0,  0, -4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[ 0, -4, 14, -4,  0,  0,  0,  0,  0,  0, -4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[ 0,  0, -4, 14, -4,  0,  0,  0,  0,  0,  0, -4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[ 0,  0,  0, -4, 14, -4,  0,  0,  0,  0,  0,  0, -4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[ 0,  0,  0,  0, -4, 

In [94]:
# a = np.array(coeff3.tolist())
# np.save('matrix.npy', a)

In [95]:
# mask3

### Sample data

In [96]:
def get_replacements(mask, f, g, reference, field):
    
    # Replace boundaries with values from the reference image
    boundary = get_boundary_mask(mask)
    bindices = np.nonzero(boundary)
    
    boundary_replacements = [
        (f[idx[1]][idx[0]], reference[idx])
        for idx in zip(*bindices)
    ]
    
    # Replace all gradient values
    m = np.ones_like(mask)
    indices = np.nonzero(m)
    
    field_replacements = [
        (g[idx[1]][idx[0]], field[idx])
        for idx in zip(*indices)
    ]
    
    return boundary_replacements + field_replacements

#### Rectangular case

In [98]:
source1 = np.array([
    [502, 527, 545, 517, 518, 492, 457, 562, 405, 420],
    [605, 512, 444, 473, 465, 496, 527, 445, 387, 397],
    [543, 446, 440, 393, 491, 472, 471, 417, 439, 371],
    [513, 476, 494, 448, 470, 491, 492, 443, 559, 514],
    [454, 487, 498, 471, 402, 484, 471, 377, 574, 452],
    [507, 478, 499, 484, 381, 372, 249, 333, 607, 410],
    [451, 497, 497, 392, 389, 476, 357, 366, 400, 464],
    [485, 517, 567, 531, 443, 324, 370, 408, 361, 464]
], dtype=np.uint16)

reference1 = np.array([
    [611, 573, 639, 564, 626, 588, 556, 503, 458, 461],
    [689, 559, 532, 550, 572, 601, 521, 466, 469, 437],
    [631, 530, 513, 504, 545, 516, 428, 444, 447, 430],
    [648, 566, 518, 514, 592, 537, 518, 468, 658, 559],
    [553, 587, 556, 544, 423, 574, 546, 452, 456, 387],
    [590, 598, 583, 564, 408, 389, 219, 498, 501, 479],
    [565, 572, 564, 436, 442, 638, 208, 382, 466, 455],
    [566, 545, 570, 507, 429, 378, 425, 474, 425, 466]
], dtype=np.uint16)

field1 = np.array([
    [-130.,   70.,  154.,  121.,   95., -102., -127.,  349., -268.,  107.],
    [ 177.,   26., -194.,   73., -118.,   28.,  239., -113., -138.,  -44.],
    [  57., -187.,  -17., -280.,  164.,  -61.,  -24., -130.,   22., -273.],
    [ 100.,  -36.,  114.,  -36.,   48.,   46.,   92.,  -73.,  266.,  314.],
    [-215.,   42.,   41.,   52., -198.,  200.,  282., -313.,  301., -150.],
    [ 111.,  -78.,   39.,  193., -123., -102., -537., -267.,  711., -422.],
    [-187.,   45.,   33., -333., -136.,  462.,  -33.,  -34., -198.,  202.],
    [-138.,   40.,  253.,  282.,  109., -419.,  -45.,  176., -169.,  248.]
], dtype=np.float)

In [99]:
replacements1 = get_replacements(mask1, f1, g1, reference1, field1)

In [100]:
# display_matrix(mat1.subs(replacements1))
mat1.subs(replacements1)

Matrix([
[                                         -1406.0],
[                                 -4*f22 + 5086.0],
[                                  -4*f32 - 992.0],
[                                 -4*f42 + 5054.0],
[                                 -4*f52 + 2510.0],
[                                 -4*f62 - 2122.0],
[                                 -4*f72 + 3722.0],
[                                           854.0],
[                                 -4*f22 + 4282.0],
[                 16*f22 - 4*f23 - 4*f32 - 6164.0],
[        -4*f22 + 16*f32 - 4*f33 - 4*f42 + 3016.0],
[        -4*f32 + 16*f42 - 4*f43 - 4*f52 - 6556.0],
[         -4*f42 + 16*f52 - 4*f53 - 4*f62 - 572.0],
[        -4*f52 + 16*f62 - 4*f63 - 4*f72 - 1140.0],
[                -4*f62 + 16*f72 - 4*f73 - 2324.0],
[                                   -4*f72 + 28.0],
[                                 -4*f23 + 2740.0],
[        -4*f22 + 16*f23 - 4*f24 - 4*f33 - 4280.0],
[ -4*f23 - 4*f32 + 16*f33 - 4*f34 - 4*f43 + 312.0],
[ -

#### Non-rectangular small case

In [108]:
reference2 = np.array([
    [555, 555, 514, 514, 514, 514, 479, 479, 479],
    [555, 555, 514, 514, 514, 514, 479, 479, 479],
    [555, 555, 514, 514, 514, 514, 479, 479, 479],
    [504, 504, 472, 472, 472, 472, 458, 458, 458],
    [504, 504, 472, 472, 472, 472, 458, 458, 458],
    [504, 504, 472, 472, 472, 472, 458, 458, 458]
], dtype=np.uint16)

field2 = np.array([
    [869,  517,  665,  618,  790,  599,  537,  616, 1287],
    [347,   -8,  -63,   -6,  -83,  -64,  -43,  186,  676],
    [425, -107,  -18,   96, -157,  -80, -118,   -6,  425],
    [443,  346,   57,   69,  -22,  -69,  -13,  -67,  596],
    [406,  -24,   47,   40,  -56,   54,   90,   25,  589],
    [1193, 1006,  714,  513,  712,  520,  579,  460,  942]
], dtype=np.int32)

In [109]:
replacements2 = get_replacements(mask2, f2, g2, reference1, field2)

In [111]:
display_matrix(mat2.subs(replacements2))

#### Big ole case

In [15]:
with rio.drivers():
    with rio.open('test2/landsat.tif') as src:
        reference = src.read(1)
    with rio.open('test2/20150705_014439_081f_analytic.tif') as src:
        source = src.read(1)
        field = (pyamg.gallery.poisson(source.shape) * source.ravel()).reshape(source.shape)

In [16]:
replacements3 = get_replacements(mask3, f3, g3, reference, field)

In [43]:
display_matrix(mat3.subs(replacements3))

In [80]:
sympy.diff(eq3, f3[12][5]).subs(symbolic_replacements3)

In [49]:
display_matrix(mat3.subs(symbolic_replacements3))

In [23]:
mat3s = mat3.subs(replacements3)

In [24]:
mat3s

Matrix([
[                                          -4*f116 + 3480.0],
[                                          -4*f126 + 2786.0],
[                                          -4*f136 + 3458.0],
[                                          -4*f146 + 3446.0],
[                                          -4*f156 - 1592.0],
[                                          -4*f166 + 2488.0],
[                                          -4*f176 + 3922.0],
[                                           -4*f186 + 418.0],
[                                            -4*f97 + 472.0],
[                                 -4*f107 - 4*f116 + 9340.0],
[                        16*f116 - 4*f117 - 4*f126 - 7582.0],
[              -4*f116 + 16*f126 - 4*f127 - 4*f136 - 4774.0],
[              -4*f126 + 16*f136 - 4*f137 - 4*f146 - 1420.0],
[              -4*f136 + 16*f146 - 4*f147 - 4*f156 - 1582.0],
[              -4*f146 + 16*f156 - 4*f157 - 4*f166 - 2808.0],
[               -4*f156 + 16*f166 - 4*f167 - 4*f176 + 702.0],

In [26]:
vec3

[f115,
 f125,
 f135,
 f145,
 f155,
 f165,
 f175,
 f185,
 f96,
 f106,
 f116,
 f126,
 f136,
 f146,
 f156,
 f166,
 f176,
 f186,
 f196,
 f206,
 f87,
 f97,
 f107,
 f117,
 f127,
 f137,
 f147,
 f157,
 f167,
 f177,
 f187,
 f197,
 f207,
 f217,
 f78,
 f88,
 f98,
 f108,
 f118,
 f128,
 f138,
 f148,
 f158,
 f168,
 f178,
 f188,
 f198,
 f208,
 f218,
 f228,
 f69,
 f79,
 f89,
 f99,
 f109,
 f119,
 f129,
 f139,
 f149,
 f159,
 f169,
 f179,
 f189,
 f199,
 f209,
 f219,
 f229,
 f510,
 f610,
 f710,
 f810,
 f910,
 f1010,
 f1110,
 f1210,
 f1310,
 f1410,
 f1510,
 f1610,
 f1710,
 f1810,
 f1910,
 f2010,
 f2110,
 f2210,
 f2310,
 f511,
 f611,
 f711,
 f811,
 f911,
 f1011,
 f1111,
 f1211,
 f1311,
 f1411,
 f1511,
 f1611,
 f1711,
 f1811,
 f1911,
 f2011,
 f2111,
 f2211,
 f2311,
 f512,
 f612,
 f712,
 f812,
 f912,
 f1012,
 f1112,
 f1212,
 f1312,
 f1412,
 f1512,
 f1612,
 f1712,
 f1812,
 f1912,
 f2012,
 f2112,
 f2212,
 f2312,
 f2412,
 f413,
 f513,
 f613,
 f713,
 f813,
 f913,
 f1013,
 f1113,
 f1213,
 f1313,
 f1413,
 f1513,
 f

In [29]:
# x1 = sympy.solve(mat1.subs(replacements1))
# x2 = sympy.solve(mat2.subs(replacements2))
x3 = sympy.solve(mat3s)

In [30]:
x3

[]

In [21]:
def apply_blended_pixels(x, reference, f):
    img = np.copy(reference)
    
    height, width = reference.shape
    
    for j in range(height):
        for i in range(width):
            if x.has_key(f[i][j]):
                img[j][i] = x[f[i][j]]
    
    return img

In [22]:
x3

[]

In [52]:
# img1 = apply_blended_pixels(x1, reference, f1)
# img2 = apply_blended_pixels(x2, reference, f2)
img3 = apply_blended_pixels(x3, reference, f3)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12, 4))
ax1.imshow(img1, vmin=300, vmax=750)
ax2.imshow(img2, vmin=300, vmax=750)

In [ ]:
print img1

In [ ]:
print img2